# Kubernetes Demo

**Acknowldegement**

This demo basically follows the [official tutorial](https://kubernetes.io/docs/tutorials/kubernetes-basics/).
Pictures are from the official documentation and are distributed under [CC BY 4.0](https://github.com/kubernetes/website/blob/master/LICENSE).

Kubernetes is an open-source system for automating deployment, scaling, and management of containerized applications.

## Kubernetes cluster

A Kubernetes cluster consists of two types of resources:
- The Master coordinates the cluster
- Nodes are the workers that run applications

Each node includes:
- kubelet agent
- container runtime

![](module_01_cluster.svg "cluster")

### Minikube

[Minikube](https://github.com/kubernetes/minikube) runs a single-node Kubernetes cluster inside a VM on your laptop

In [1]:
minikube start

Starting local Kubernetes v1.8.0 cluster...
Starting VM...
Getting VM IP address...
Moving files into cluster...
Setting up certs...
Connecting to cluster...
Setting up kubeconfig...
Starting cluster components...
Kubectl is now configured to use the cluster.
Loading cached images from config file.


In [2]:
minikube status

minikube: Running
cluster: Running
kubectl: Correctly Configured: pointing to minikube-vm at 192.168.99.100


Access minikube via a GUI

In [3]:
minikube dashboard

Opening kubernetes dashboard in default browser...


Access minikube via CLI with kubectl

In [4]:
kubectl version

Client Version: version.Info{Major:"1", Minor:"8", GitVersion:"v1.8.4", GitCommit:"9befc2b8928a9426501d3bf62f72849d5cbcd5a3", GitTreeState:"clean", BuildDate:"2017-11-20T19:11:22Z", GoVersion:"go1.9.2", Compiler:"gc", Platform:"darwin/amd64"}
Server Version: version.Info{Major:"1", Minor:"8", GitVersion:"v1.8.0", GitCommit:"0b9efaeb34a2fc51ff8e4d34ad9bc6375459c4a4", GitTreeState:"clean", BuildDate:"2017-11-29T22:43:34Z", GoVersion:"go1.9.1", Compiler:"gc", Platform:"linux/amd64"}


In [5]:
kubectl cluster-info

Kubernetes master is running at https://192.168.99.100:8443

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [6]:
kubectl get nodes

NAME       STATUS    ROLES     AGE       VERSION
minikube   Ready     <none>    1d        v1.8.0


In [7]:
kubectl config view

apiVersion: v1
clusters:
- cluster:
    certificate-authority: /Users/benjaminbertrand/.minikube/ca.crt
    server: https://192.168.99.100:8443
  name: minikube
contexts:
- context:
    cluster: minikube
    user: minikube
  name: minikube
current-context: minikube
kind: Config
preferences: {}
users:
- name: minikube
  user:
    client-certificate: /Users/benjaminbertrand/.minikube/client.crt
    client-key: /Users/benjaminbertrand/.minikube/client.key


Access minikube via API

In [8]:
curl https://192.168.99.100:8443 --insecure

{
  "kind": "Status",
  "apiVersion": "v1",
  "metadata": {
    
  },
  "status": "Failure",
  "message": "Unauthorized",
  "reason": "Unauthorized",
  "code": 401
}

In [9]:
# Run kubectl proxy in a terminal

In [10]:
curl http://localhost:8001/

{
  "paths": [
    "/api",
    "/api/v1",
    "/apis",
    "/apis/",
    "/apis/admissionregistration.k8s.io",
    "/apis/admissionregistration.k8s.io/v1alpha1",
    "/apis/apiextensions.k8s.io",
    "/apis/apiextensions.k8s.io/v1beta1",
    "/apis/apiregistration.k8s.io",
    "/apis/apiregistration.k8s.io/v1beta1",
    "/apis/apps",
    "/apis/apps/v1beta1",
    "/apis/apps/v1beta2",
    "/apis/authentication.k8s.io",
    "/apis/authentication.k8s.io/v1",
    "/apis/authentication.k8s.io/v1beta1",
    "/apis/authorization.k8s.io",
    "/apis/authorization.k8s.io/v1",
    "/apis/authorization.k8s.io/v1beta1",
    "/apis/autoscaling",
    "/apis/autoscaling/v1",
    "/apis/autoscaling/v2beta1",
    "/apis/batch",
    "/apis/batch/v1",
    "/apis/batch/v1beta1",
    "/apis/batch/v2alpha1",
    "/apis/certificates.k8s.io",
    "/apis/certificates.k8s.io/v1beta1",
    "/apis/extensions",
    "/apis/extensions/v1beta1",
    "/apis/networking.k8s.io",
    "/apis/networking.k8s.io/v1",
    "/

## Deployment

A **deployment** instructs Kubernetes how to create and update instances of your application

![](module_02_first_app.svg "Deployment")

Application instances are hosted inside a **Pod**. A Pod is the smallest deployable object in the Kubernetes object model.
A Pod represents a group of one or more application containers and some shared resources (storage, networking).

![](module_03_pods.svg "pods")

A Pod always runs on a **Node**

![](module_03_nodes.svg)

#### Let's deploy the *microbot* web application

##### With docker we would just run:

In [11]:
docker run -d -p 8080:80 dontrebootme/microbot:v1

62a4a36293bbe2a5c95079592604717ce972fd514e22bfa72e877a040d4fb344


In [12]:
docker ps

CONTAINER ID        IMAGE                      COMMAND                  CREATED             STATUS              PORTS                  NAMES
62a4a36293bb        dontrebootme/microbot:v1   "/bin/sh -c /start..."   9 seconds ago       Up 8 seconds        0.0.0.0:8080->80/tcp   compassionate_haibt


In [13]:
docker stop compassionate_haibt

compassionate_haibt


##### With Kubernetes

In [14]:
kubectl run kubernetes-microbot --image=dontrebootme/microbot:v1 --port 80

deployment "kubernetes-microbot" created


In [15]:
kubectl get deployments

NAME                  DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
kubernetes-microbot   1         1         1            1           3s


In [16]:
kubectl get pods

NAME                                   READY     STATUS    RESTARTS   AGE
kubernetes-microbot-7b78749599-ccn6l   1/1       Running   0          7s


In [17]:
POD_NAME=kubernetes-microbot-7b78749599-ccn6l

We can get more information about the Pod

In [18]:
kubectl describe pods

Name:           kubernetes-microbot-7b78749599-ccn6l
Namespace:      default
Node:           minikube/192.168.99.100
Start Time:     Mon, 04 Dec 2017 11:20:15 +0100
Labels:         pod-template-hash=3634305155
                run=kubernetes-microbot
Annotations:    kubernetes.io/created-by={"kind":"SerializedReference","apiVersion":"v1","reference":{"kind":"ReplicaSet","namespace":"default","name":"kubernetes-microbot-7b78749599","uid":"b80cb235-d8dc-11e7-a20d-0...
Status:         Running
IP:             172.17.0.4
Created By:     ReplicaSet/kubernetes-microbot-7b78749599
Controlled By:  ReplicaSet/kubernetes-microbot-7b78749599
Containers:
  kubernetes-microbot:
    Container ID:   docker://7ef945bd049a3014e1da91d51ba1d92607cf53b3b43db97e3eb7a4312ecded50
    Image:          dontrebootme/microbot:v1
    Image ID:       docker-pullable://dontrebootme/microbot@sha256:ec71565e9cad762bdbd3ecc382b26162f07cea81a803861bc1e5099758e46ee0
    Port:           80/TCP
    State:          Running
  

In [19]:
kubectl logs $POD_NAME

We can execute commands inside the Pod

In [20]:
kubectl exec $POD_NAME env

PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
HOSTNAME=kubernetes-microbot-7b78749599-ccn6l
KUBERNETES_SERVICE_PORT_HTTPS=443
KUBERNETES_PORT=tcp://10.96.0.1:443
KUBERNETES_PORT_443_TCP=tcp://10.96.0.1:443
KUBERNETES_PORT_443_TCP_PROTO=tcp
KUBERNETES_PORT_443_TCP_PORT=443
KUBERNETES_PORT_443_TCP_ADDR=10.96.0.1
KUBERNETES_SERVICE_HOST=10.96.0.1
KUBERNETES_SERVICE_PORT=443
HOME=/root


In [21]:
kubectl exec -it $POD_NAME ls /

bin             lib             proc            start_nginx.sh  var
dev             linuxrc         root            sys
etc             media           run             tmp
home            mnt             sbin            usr


In [22]:
kubectl exec -it $POD_NAME cat /start_nginx.sh

#!/bin/sh
hostname=`hostname -f`
sed -i "s/XXX/${hostname}/" /usr/share/nginx/html/index.html
/usr/sbin/nginx


How to access our app?

By default Pods are only visible from other pods and services within the same cluster.

We can access Pods via the API (an endpoint is automatically created by the API server):

In [23]:
curl http://localhost:8001/api/v1/proxy/namespaces/default/pods/$POD_NAME/

<!DOCTYPE html>
<html>
  <style type="text/css">
    .centered
      {
      text-align:center;
      margin-top:0px;
      margin-bottom:0px;
      padding:0px;
      }
  </style>
  <body>
    <p class="centered"><img src="microbot.png" alt="microbot"/></p>
    <p class="centered">Container hostname: kubernetes-microbot-7b78749599-ccn6l</p>
  </body>
</html>


To expose the application to the ouside world, we have to create a **Service**

## Service

- abstraction to define logical set of Pods and a policy by which to access them
- route traffic across a set of Pods
- match Pods using labels and selectors

ServiceType:
- ClusterIP (default): only reachable within the cluster
- NodePort: Service accessible from outside the cluster using *NodeIP:NodePort*
- LoadBalancer
- ExternalName
    
![](module_04_services.svg)

![](module_04_labels.svg)

In [25]:
kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   1d


Create a Service to expose microbot using the NodePort Service type

In [26]:
kubectl expose deployment/kubernetes-microbot --type="NodePort" --port 80

service "kubernetes-microbot" exposed


In [28]:
kubectl get services

NAME                  TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
kubernetes            ClusterIP   10.96.0.1       <none>        443/TCP        1d
kubernetes-microbot   NodePort    10.105.40.107   <none>        80:32303/TCP   7s


In [29]:
kubectl describe services/kubernetes-microbot

Name:                     kubernetes-microbot
Namespace:                default
Labels:                   run=kubernetes-microbot
Annotations:              <none>
Selector:                 run=kubernetes-microbot
Type:                     NodePort
IP:                       10.105.40.107
Port:                     <unset>  80/TCP
TargetPort:               80/TCP
NodePort:                 <unset>  32303/TCP
Endpoints:                172.17.0.4:80
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


In [30]:
NODE_PORT=32303

In [31]:
minikube ip

192.168.99.100


In [32]:
curl http://$(minikube ip):$NODE_PORT

<!DOCTYPE html>
<html>
  <style type="text/css">
    .centered
      {
      text-align:center;
      margin-top:0px;
      margin-bottom:0px;
      padding:0px;
      }
  </style>
  <body>
    <p class="centered"><img src="microbot.png" alt="microbot"/></p>
    <p class="centered">Container hostname: kubernetes-microbot-7b78749599-ccn6l</p>
  </body>
</html>


A label was automatically created for our Pod

In [34]:
kubectl describe deployment

Name:                   kubernetes-microbot
Namespace:              default
CreationTimestamp:      Mon, 04 Dec 2017 11:20:15 +0100
Labels:                 run=kubernetes-microbot
Annotations:            deployment.kubernetes.io/revision=1
Selector:               run=kubernetes-microbot
Replicas:               1 desired | 1 updated | 1 total | 1 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  1 max unavailable, 1 max surge
Pod Template:
  Labels:  run=kubernetes-microbot
  Containers:
   kubernetes-microbot:
    Image:        dontrebootme/microbot:v1
    Port:         80/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
OldReplicaSets:  <none>
NewReplicaSet:   kubernetes-microbot-7b78749599 (1/1 replicas created)
Events:
  Type    Reason             Age   From         

Use the label to get the list of pods or services

In [35]:
kubectl get pods -l run=kubernetes-microbot

NAME                                   READY     STATUS    RESTARTS   AGE
kubernetes-microbot-7b78749599-ccn6l   1/1       Running   0          5m


In [36]:
kubectl get services -l run=kubernetes-microbot

NAME                  TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
kubernetes-microbot   NodePort   10.105.40.107   <none>        80:32303/TCP   1m


In [37]:
kubectl label pod $POD_NAME app=v1

pod "kubernetes-microbot-7b78749599-ccn6l" labeled


In [39]:
kubectl describe pods $POD_NAME

Name:           kubernetes-microbot-7b78749599-ccn6l
Namespace:      default
Node:           minikube/192.168.99.100
Start Time:     Mon, 04 Dec 2017 11:20:15 +0100
Labels:         app=v1
                pod-template-hash=3634305155
                run=kubernetes-microbot
Annotations:    kubernetes.io/created-by={"kind":"SerializedReference","apiVersion":"v1","reference":{"kind":"ReplicaSet","namespace":"default","name":"kubernetes-microbot-7b78749599","uid":"b80cb235-d8dc-11e7-a20d-0...
Status:         Running
IP:             172.17.0.4
Created By:     ReplicaSet/kubernetes-microbot-7b78749599
Controlled By:  ReplicaSet/kubernetes-microbot-7b78749599
Containers:
  kubernetes-microbot:
    Container ID:   docker://7ef945bd049a3014e1da91d51ba1d92607cf53b3b43db97e3eb7a4312ecded50
    Image:          dontrebootme/microbot:v1
    Image ID:       docker-pullable://dontrebootme/microbot@sha256:ec71565e9cad762bdbd3ecc382b26162f07cea81a803861bc1e5099758e46ee0
    Port:           80/TCP
    Sta

In [40]:
curl http://$(minikube ip):$NODE_PORT

<!DOCTYPE html>
<html>
  <style type="text/css">
    .centered
      {
      text-align:center;
      margin-top:0px;
      margin-bottom:0px;
      padding:0px;
      }
  </style>
  <body>
    <p class="centered"><img src="microbot.png" alt="microbot"/></p>
    <p class="centered">Container hostname: kubernetes-microbot-7b78749599-ccn6l</p>
  </body>
</html>


If we delete the service, the app won't be accessible anymore from the outside world

In [41]:
kubectl delete service -l run=kubernetes-microbot

service "kubernetes-microbot" deleted


In [42]:
kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   1d


In [43]:
curl http://$(minikube ip):$NODE_PORT

curl: (7) Failed to connect to 192.168.99.100 port 32303: Connection refused


: 7

The Pod is still running and can be accessed by the API server

In [44]:
kubectl get pods

NAME                                   READY     STATUS    RESTARTS   AGE
kubernetes-microbot-7b78749599-ccn6l   1/1       Running   0          6m


In [45]:
curl http://localhost:8001/api/v1/proxy/namespaces/default/pods/$POD_NAME/

<!DOCTYPE html>
<html>
  <style type="text/css">
    .centered
      {
      text-align:center;
      margin-top:0px;
      margin-bottom:0px;
      padding:0px;
      }
  </style>
  <body>
    <p class="centered"><img src="microbot.png" alt="microbot"/></p>
    <p class="centered">Container hostname: kubernetes-microbot-7b78749599-ccn6l</p>
  </body>
</html>


Let's restart the service

In [46]:
kubectl expose deployment/kubernetes-microbot --type="NodePort" --port 80

service "kubernetes-microbot" exposed


## Scaling

Kubernetes allows to easily scale an application by starting more Pods.

![](module_05_scaling1.svg)

![](module_05_scaling2.svg)

In [47]:
kubectl get deployments

NAME                  DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
kubernetes-microbot   1         1         1            1           7m


In [48]:
kubectl scale deployments/kubernetes-microbot --replicas=4

deployment "kubernetes-microbot" scaled


In [49]:
kubectl get deployments

NAME                  DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
kubernetes-microbot   4         4         4            4           7m


In [50]:
kubectl get pods -o wide

NAME                                   READY     STATUS    RESTARTS   AGE       IP           NODE
kubernetes-microbot-7b78749599-bjknr   1/1       Running   0          6s        172.17.0.7   minikube
kubernetes-microbot-7b78749599-ccn6l   1/1       Running   0          8m        172.17.0.4   minikube
kubernetes-microbot-7b78749599-dfzlj   1/1       Running   0          6s        172.17.0.5   minikube
kubernetes-microbot-7b78749599-khhmj   1/1       Running   0          6s        172.17.0.6   minikube


In [51]:
kubectl describe deployments/kubernetes-microbot

Name:                   kubernetes-microbot
Namespace:              default
CreationTimestamp:      Mon, 04 Dec 2017 11:20:15 +0100
Labels:                 run=kubernetes-microbot
Annotations:            deployment.kubernetes.io/revision=1
Selector:               run=kubernetes-microbot
Replicas:               4 desired | 4 updated | 4 total | 4 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  1 max unavailable, 1 max surge
Pod Template:
  Labels:  run=kubernetes-microbot
  Containers:
   kubernetes-microbot:
    Image:        dontrebootme/microbot:v1
    Port:         80/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
OldReplicaSets:  <none>
NewReplicaSet:   kubernetes-microbot-7b78749599 (4/4 replicas created)
Events:
  Type    Reason             Age   From         

In [52]:
kubectl describe services/kubernetes-microbot

Name:                     kubernetes-microbot
Namespace:                default
Labels:                   run=kubernetes-microbot
Annotations:              <none>
Selector:                 run=kubernetes-microbot
Type:                     NodePort
IP:                       10.106.71.208
Port:                     <unset>  80/TCP
TargetPort:               80/TCP
NodePort:                 <unset>  30848/TCP
Endpoints:                172.17.0.4:80,172.17.0.5:80,172.17.0.6:80 + 1 more...
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


In [53]:
NODE_PORT=30848

In [54]:
minikube ip

192.168.99.100


In [57]:
curl http://$(minikube ip):$NODE_PORT

<!DOCTYPE html>
<html>
  <style type="text/css">
    .centered
      {
      text-align:center;
      margin-top:0px;
      margin-bottom:0px;
      padding:0px;
      }
  </style>
  <body>
    <p class="centered"><img src="microbot.png" alt="microbot"/></p>
    <p class="centered">Container hostname: kubernetes-microbot-7b78749599-dfzlj</p>
  </body>
</html>


We can also scale down

In [58]:
kubectl scale deployments/kubernetes-microbot --replicas=2

deployment "kubernetes-microbot" scaled


In [59]:
kubectl get deployments

NAME                  DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
kubernetes-microbot   2         2         2            2           9m


In [60]:
kubectl get pods -o wide

NAME                                   READY     STATUS        RESTARTS   AGE       IP           NODE
kubernetes-microbot-7b78749599-bjknr   1/1       Terminating   0          1m        172.17.0.7   minikube
kubernetes-microbot-7b78749599-ccn6l   1/1       Running       0          9m        172.17.0.4   minikube
kubernetes-microbot-7b78749599-dfzlj   1/1       Running       0          1m        172.17.0.5   minikube
kubernetes-microbot-7b78749599-khhmj   1/1       Terminating   0          1m        172.17.0.6   minikube


### Rolling update

In [61]:
kubectl scale deployments/kubernetes-microbot --replicas=4

deployment "kubernetes-microbot" scaled


In [63]:
kubectl get pods -o wide

NAME                                   READY     STATUS        RESTARTS   AGE       IP           NODE
kubernetes-microbot-7b78749599-9s4s2   1/1       Running       0          5s        172.17.0.7   minikube
kubernetes-microbot-7b78749599-bjknr   0/1       Terminating   0          2m        <none>       minikube
kubernetes-microbot-7b78749599-ccn6l   1/1       Running       0          10m       172.17.0.4   minikube
kubernetes-microbot-7b78749599-dfzlj   1/1       Running       0          2m        172.17.0.5   minikube
kubernetes-microbot-7b78749599-khhmj   0/1       Terminating   0          2m        <none>       minikube
kubernetes-microbot-7b78749599-mdrgr   1/1       Running       0          5s        172.17.0.6   minikube


In [64]:
kubectl describe pods

Name:           kubernetes-microbot-7b78749599-9s4s2
Namespace:      default
Node:           minikube/192.168.99.100
Start Time:     Mon, 04 Dec 2017 11:30:46 +0100
Labels:         pod-template-hash=3634305155
                run=kubernetes-microbot
Annotations:    kubernetes.io/created-by={"kind":"SerializedReference","apiVersion":"v1","reference":{"kind":"ReplicaSet","namespace":"default","name":"kubernetes-microbot-7b78749599","uid":"b80cb235-d8dc-11e7-a20d-0...
Status:         Running
IP:             172.17.0.7
Created By:     ReplicaSet/kubernetes-microbot-7b78749599
Controlled By:  ReplicaSet/kubernetes-microbot-7b78749599
Containers:
  kubernetes-microbot:
    Container ID:   docker://fb06d07d7cd7ce052c8a0f492eac087ca29e24f661a66379e0b2b9ddc402e512
    Image:          dontrebootme/microbot:v1
    Image ID:       docker-pullable://dontrebootme/microbot@sha256:ec71565e9cad762bdbd3ecc382b26162f07cea81a803861bc1e5099758e46ee0
    Port:           80/TCP
    State:          Running
  

    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-dp5x6 (ro)
Conditions:
  Type           Status
  Initialized    True 
  Ready          True 
  PodScheduled   True 
Volumes:
  default-token-dp5x6:
    Type:        Secret (a volume populated by a Secret)
    SecretName:  default-token-dp5x6
    Optional:    false
QoS Class:       BestEffort
Node-Selectors:  <none>
Tolerations:     <none>
Events:
  Type    Reason                 Age   From               Message
  ----    ------                 ----  ----               -------
  Normal  Scheduled              2m    default-scheduler  Successfully assigned kubernetes-microbot-7b78749599-dfzlj to minikube
  Normal  SuccessfulMountVolume  2m    kubelet, minikube  MountVolume.SetUp succeeded for volume "default-token-dp5x6"
  Normal  Pulled                 2m    kubelet, minikube  Container image "dontrebootme/microbot:v1" already present on machine
  Normal  Created                

Let's update microbot to v2

In [65]:
kubectl set image deployments/kubernetes-microbot kubernetes-microbot=dontrebootme/microbot:v2

deployment "kubernetes-microbot" image updated


In [66]:
kubectl get pods -o wide

NAME                                   READY     STATUS        RESTARTS   AGE       IP           NODE
kubernetes-microbot-7b78749599-9s4s2   1/1       Terminating   0          35s       172.17.0.7   minikube
kubernetes-microbot-7b78749599-bjknr   0/1       Terminating   0          3m        <none>       minikube
kubernetes-microbot-7b78749599-ccn6l   1/1       Running       0          11m       172.17.0.4   minikube
kubernetes-microbot-7b78749599-dfzlj   1/1       Terminating   0          3m        172.17.0.5   minikube
kubernetes-microbot-7b78749599-khhmj   0/1       Terminating   0          3m        <none>       minikube
kubernetes-microbot-7b78749599-mdrgr   1/1       Terminating   0          35s       172.17.0.6   minikube
kubernetes-microbot-7f9f48b956-2w6dj   1/1       Running       0          1s        172.17.0.9   minikube
kubernetes-microbot-7f9f48b956-j6z8l   0/1       Pending       0          0s        <none>       minikube
kubernetes-microbot-7f9f48b956-qlmc6   1/1       R

In [67]:
kubectl get deployments

NAME                  DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
kubernetes-microbot   4         4         4            4           11m


In [68]:
kubectl rollout status deployments/kubernetes-microbot

deployment "kubernetes-microbot" successfully rolled out


Let's try to update to v10

In [69]:
kubectl set image deployments/kubernetes-microbot kubernetes-microbot=dontrebootme/microbot:v10

deployment "kubernetes-microbot" image updated


In [70]:
kubectl get deployments

NAME                  DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
kubernetes-microbot   4         5         2            3           11m


In [72]:
kubectl get pods

NAME                                   READY     STATUS         RESTARTS   AGE
kubernetes-microbot-7b78749599-9s4s2   0/1       Terminating    0          1m
kubernetes-microbot-7b78749599-ccn6l   0/1       Terminating    0          12m
kubernetes-microbot-7b78749599-dfzlj   0/1       Terminating    0          4m
kubernetes-microbot-7b78749599-mdrgr   0/1       Terminating    0          1m
kubernetes-microbot-7f9f48b956-2w6dj   1/1       Running        0          1m
kubernetes-microbot-7f9f48b956-j6z8l   0/1       Terminating    0          1m
kubernetes-microbot-7f9f48b956-qlmc6   1/1       Running        0          1m
kubernetes-microbot-7f9f48b956-t4cs5   1/1       Running        0          1m
kubernetes-microbot-8658647c4c-84mw9   0/1       ErrImagePull   0          42s
kubernetes-microbot-8658647c4c-tfj9w   0/1       ErrImagePull   0          42s


There is no image v10. Let's roll back.

In [73]:
kubectl rollout undo deployments/kubernetes-microbot

deployment "kubernetes-microbot" rolled back


In [74]:
kubectl get deployments

NAME                  DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
kubernetes-microbot   4         4         4            4           12m


In [75]:
kubectl get pods

NAME                                   READY     STATUS        RESTARTS   AGE
kubernetes-microbot-7f9f48b956-2w6dj   1/1       Running       0          1m
kubernetes-microbot-7f9f48b956-qlmc6   1/1       Running       0          1m
kubernetes-microbot-7f9f48b956-t4cs5   1/1       Running       0          1m
kubernetes-microbot-7f9f48b956-wkxr9   1/1       Running       0          8s
kubernetes-microbot-8658647c4c-84mw9   0/1       Terminating   0          57s


We are back to the v2 image